In [2]:
from pathlib import Path
from wsireg.wsireg import WsiReg2D

## wsireg for multi-modal image registration of whole slide images

First, we will initialize the graph model and add modalities. Each modality is loaded with 3
pieces of information:
* the file path to image file to be loaded
* the image spatial resolution,
* the preprocessing parameters.

Images that are to be registered must be a single plane and
the preprocessing parameters take care of this.

Two default processes are available:
* _FL_ : take multi-channel image and converts to single channel through maximum intensity projection
* _BF_ : assumed to be 24-bit RGB images, these are converted to a single channel by taking RGB to greyscale
then intensity is inverted so background is black and foreground is white.


In [3]:
# we define a project name, output directory and whether to cache
# preprocessed images when making a registration graph instance
# project name = Sample107
# output directory is same as the data
# cache_images is True, this saves time during registration and avoids unnecessary io
reg_graph = WsiReg2D("Sample107","./", cache_images=True)

We will add multiple image modalities (7 in total) and 6 of these will be transformed to a single target but will take different paths.
The target modality is a post-imaging mass spectrometry(IMS) autofluorescence (AF) image to which IMS data has been previously registered.
Other modalities are AF of this section before IMS, AF on a serial section prior to immunofluorescence (IF). Finally, 3 cycles
of IF on a single section.

In [4]:
# global target modality
# preprocessing
# ch_indicices selects certain channels
# as_uint8 bytescales data to unsigned 8 bit for memory saving
# contrast_enhance makes low contrast fluorescence images brighter for registration
# if a value in the processing dict is set to None it is skipped. These are the defaults

reg_graph.add_modality(
    "postAF_IMS",
    "C:/biomic/devdata/S107_postAF_IMS.tif",
    image_res=1,
    prepro_dict={
        "image_type": "FL",
        "ch_indices": None,
        "as_uint8": True,
        "contrast_enhance": None,
    },
)

In [5]:
# this is an AF modality
# we select the first channel for registration
# scale intensity to uint8
# and rotate the image 180 degrees counter-clockwise using "rot_cc"
# very large rotations are hard to capture with intensity based registration
# therefore we use prior knowledge to add this information

reg_graph.add_modality(
    "preAF_IMS",
    "C:/biomic/devdata/S107_preAF_IMS.czi",
    image_res=0.65,
    prepro_dict={
        "image_type": "FL",
        "ch_indices": [0],
        "as_uint8": True,
        "rot_cc": 180,
    },
)




In [6]:
# serial section AF

# in addition the previous modalities pre-processing
# this modality's section is "mirror" to the previous and thus needs
# a coordinate flip so we use "flip" preprocessing and flip horizontally the image

reg_graph.add_modality(
    "preAF_MxIF",
    "C:/biomic/devdata/S107_preAF_MxIF.czi",
    image_res=0.65,
    prepro_dict={
        "image_type": "FL",
        "ch_indices": [0],
        "as_uint8": True,
        "rot_cc": 90,
        "flip": "h",
    },
)

### IF registration
The multiple cycles of IF are loaded as modalities selecting their DAPI channel to achieve cell nucleus-to-nucleus
registration. However, MxIF cycle 1 (modality : MxIF_cyc1) will be registered to this sections corresponding AF image.
Because of this, we will use special preprocessing for that registration described later.

In [8]:
# all MxIF cycles have similar spatial pre-processing

reg_graph.add_modality(
    "MxIF_cyc1",
    "C:/biomic/devdata/S107_MxIF_Cyc1.czi",
    image_res=0.65,
    prepro_dict={
        "image_type": "FL",
        "ch_indices": [0],
        "as_uint8": True,
        "rot_cc": 90,
        "flip": "h",
    },
)

reg_graph.add_modality(
    "MxIF_cyc2",
    "C:/biomic/devdata/S107_MxIF_Cyc2.czi",
    image_res=0.65,
    prepro_dict={
        "image_type": "FL",
        "ch_indices": [0],
        "as_uint8": True,
        "rot_cc": 90,
        "flip": "h",
    },
)

reg_graph.add_modality(
    "MxIF_cyc3",
    "C:/biomic/devdata/S107_MxIF_Cyc3.czi",
    image_res=0.65,
    prepro_dict={
        "image_type": "FL",
        "ch_indices": [0],
        "as_uint8": True,
        "rot_cc": 90,
        "flip": "h",
    },
)



### defining the registrations (edges) of the graph

Now we will define how modalities move through the graph. For every modality to be registered we define
three settings:
* modality to be registered
* modality to which it will be registered
* through modalities, modalities which will form the path of alignment from one image to another
* registration parameters (elastix parameters)
* whether to use special preprocessing for this particular registration

For this case where there are serial sections, we want to define registration paths that use the best information
for alignment. For instance, we have AF images on the two serial sections so complex non-linear registration is best performed mono-modally.
The alternative would be to register the IF data directly to the AF image, where the specificty of IF would not help find commonalities between the modalities
on which to register.

In [9]:
# register preAF_IMS to postAF_IMS
reg_graph.add_reg_path(
    "preAF_IMS", "postAF_IMS", thru_modality=None, reg_params=["rigid"]
)

# register preAF_MxIF to postAF_IMS
# BUT go through preAF_IMS (same section)
# this will align preAF_MXiF to preAF_IMS then
# append the transformations from preAF_IMS to postAF_IMS
# in this cases we use both rigid and non-linear ("nl") elastix transformation models
# for default we have "rigid","affine" and "nl" these work in 90% of the cases
# alternatively you can add a file path string to elastix registration parameters saved in a text file
reg_graph.add_reg_path(
    "preAF_MxIF", "postAF_IMS", thru_modality="preAF_IMS", reg_params=["rigid", "nl"],
)

In [10]:
# here we register MxIF_cyc1 to postAF_IMS through preAF_MxIF
# the graph will find the necessary edges to follow to get from MxIF_cyc1 to postAF_IMS
# but we use special preprocessing here since the set preprocessing for MxIF_cyc1 is
# to select the DAPI nuclei channel but here we will use the maximum intensity projection
# to process the 4 channels to 1 to align to AF
# this increases the amount of texture and spatial information for registration
# as DAPI is spatially sparse in signal, compared to all channels

reg_graph.add_reg_path(
    "MxIF_cyc1",
    "postAF_IMS",
    thru_modality="preAF_MxIF",
    reg_params=["rigid"],
    override_prepro={
        "source": {
            "image_type": "FL",
            "ch_indices": None,
            "as_uint8": True,
            "rot_cc": 90,
            "flip": "h",
        },
        "target": None,
    },
)



In [11]:
# now we align MxIF cycles 2 and 3 to postAF, this time going through MxIF Cyc1
# MxIF_cyc1 goes through preAF_MxIF to preAF_IMS to postAF_IMS
reg_graph.add_reg_path(
    "MxIF_cyc2", "postAF_IMS", thru_modality="MxIF_cyc1", reg_params=["rigid"],
)

reg_graph.add_reg_path(
    "MxIF_cyc3", "postAF_IMS", thru_modality="MxIF_cyc1", reg_params=["rigid"],
)

### graph information
We can use some utility functions to learn about the graph.

In [12]:
print("number of modalities : {}".format(reg_graph.n_modalities))
print("number of registrations : {}".format(reg_graph.n_registrations))

number of modalities : 6
number of registrations : 5


In [13]:
print(reg_graph.reg_paths)

{'preAF_IMS': ['postAF_IMS'], 'preAF_MxIF': ['preAF_IMS', 'postAF_IMS'], 'MxIF_cyc1': ['preAF_MxIF', 'postAF_IMS'], 'MxIF_cyc2': ['MxIF_cyc1', 'postAF_IMS'], 'MxIF_cyc3': ['MxIF_cyc1', 'postAF_IMS']}


In [14]:
# show all transformation paths
print(reg_graph.transform_paths)


{'preAF_IMS': [{'source': 'preAF_IMS', 'target': 'postAF_IMS'}], 'preAF_MxIF': [{'source': 'preAF_MxIF', 'target': 'preAF_IMS'}, {'source': 'preAF_IMS', 'target': 'postAF_IMS'}], 'MxIF_cyc1': [{'source': 'MxIF_cyc1', 'target': 'preAF_MxIF'}, {'source': 'preAF_MxIF', 'target': 'preAF_IMS'}, {'source': 'preAF_IMS', 'target': 'postAF_IMS'}], 'MxIF_cyc2': [{'source': 'MxIF_cyc2', 'target': 'MxIF_cyc1'}, {'source': 'MxIF_cyc1', 'target': 'preAF_MxIF'}, {'source': 'preAF_MxIF', 'target': 'preAF_IMS'}, {'source': 'preAF_IMS', 'target': 'postAF_IMS'}], 'MxIF_cyc3': [{'source': 'MxIF_cyc3', 'target': 'MxIF_cyc1'}, {'source': 'MxIF_cyc1', 'target': 'preAF_MxIF'}, {'source': 'preAF_MxIF', 'target': 'preAF_IMS'}, {'source': 'preAF_IMS', 'target': 'postAF_IMS'}]}


In [15]:
# look at how MxIF_cyc3 will be transformed specifically
print(*reg_graph.transform_paths["MxIF_cyc1"], sep="\n")



{'source': 'MxIF_cyc1', 'target': 'preAF_MxIF'}
{'source': 'preAF_MxIF', 'target': 'preAF_IMS'}
{'source': 'preAF_IMS', 'target': 'postAF_IMS'}


From the above we can see a list of transformations that will be applied to transform MxIF_cyc1 to
postAF_IMS.

### executing the graph
Now we have loaded and set all registration modalities, we can begin the alignment and transformation process.

In [ ]:
# align images, this can take some time depending on computer performance
# generally 3-10 minutes per registration when using the default models
reg_graph.register_images()

# save the transformations as a json on disk (can be reloaded and used again)
reg_graph.save_transformations()

# transform and save images to disk as tifs using ITK io
reg_graph.transform_images()